In [93]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd

api_key = "rA0vy4zNu4RpkIseRL4eccSlExzx2RMphwqHRUQo"

# # Pull ORI Codes from original csv

In [94]:
project_one_df = pd.read_csv("data_for_project_one.csv")
project_one_df.head()

,State,State Ab.,State Capital,Population,ORI Code,No. of Officers,No. of (Offenses
0,ALABAMA,AL,Montgomery,"198,525",NaN,NaN,NaN
1,ALASKA,AK,Juneau,"32,113",NaN,NaN,NaN
2,ARIZONA,AZ,Phoenix,"1,680,992",NaN,NaN,NaN
3,ARKANSAS,AR,Little Rock,"197,312",NaN,NaN,NaN
4,CALIFORNIA,CA,Sacramento,"513,624",NaN,NaN,NaN


In [108]:
#states 21-30
ori = project_one_df["ORI Code"][19:28].tolist()
ori

['MD0020100',
 'MA0130100',
 'MI3351900',
 'MN0620900',
 'MS0250100',
 'MO0260100',
 'MT0250100',
 'NB0550100',
 'NV0130000']

# # Loop through API endpoint for crime case num

In [96]:
#states 21-30
begin_year = "2018"
end_year = "2018"

simple_assault_cases = []
stolen_property_cases = []
rape_cases = []
ori_list = ori


for ori2 in range(len(ori_list)):
    response2 = requests.get(f"https://api.usa.gov/crime/fbi/sapi/api/data/arrest/agencies/offense/{ori_list[ori2]}/all/{begin_year}/{end_year}?API_KEY={api_key}").json()
    try:
        simple_assault_cases.append(response2["results"][0]["simple_assault"])
        stolen_property_cases.append(response2["results"][0]["stolen_property"])
        rape_cases.append(response2["results"][0]["rape"])
    except IndexError:
        simple_assault_cases.append(np.nan)
        stolen_property_cases.append(np.nan)
        rape_cases.append(np.nan)
    # Printing queries
    print(f"Processing record {ori2 + 1}")
print("------")
print("Done")
print("------")

Processing record 1
Processing record 2
Processing record 3
Processing record 4
Processing record 5
Processing record 6
Processing record 7
Processing record 8
Processing record 9
------
Done
------


# # Merge case number data with original csv 

In [97]:
df=pd.DataFrame(list(zip(ori_list, simple_assault_cases, stolen_property_cases, rape_cases)), 
               columns = ['ORI Code', "Simple Assault Cases", "Stolen Property Cases", "Rape Cases"])


cases_merged_df = pd.merge(project_one_df, df, on= "ORI Code")

cases_merged_df.head()

,State,State Ab.,State Capital,Population,ORI Code,No. of Officers,No. of (Offenses,Simple Assault Cases,Stolen Property Cases,Rape Cases
0,MARYLAND,MD,Annapolis,"39,174",MD0020100,NaN,NaN,162,3,5
1,MASSACHUSETTS,MA,Boston,"692,600",MA0130100,NaN,NaN,1612,169,43
2,MICHIGAN,MI,Lansing,"118,210",MI3351900,NaN,NaN,791,22,17
3,MINNESOTA,MN,Saint Paul,"308,096",MN0620900,NaN,NaN,984,43,16
4,MISSISSIPPI,MS,Jackson,"160,628",MS0250100,NaN,NaN,445,106,8


# # Loop through API endpoint for number of officers per ori 

In [98]:
ori_list = ori
female =[]
male =[]
begin_year = "2018"
end_year = "2018"

print("Beginning Data Retrieval")
print("------------------------")
for ori2 in range(len(ori_list)):
    response = requests.get(f"https://api.usa.gov/crime/fbi/sapi/api/police-employment/agencies/{ori_list[ori2]}/{begin_year}/{end_year}?API_KEY={api_key}").json()

    try:
        print(f"Processing record {ori2 + 1}")
        female.append(response["results"][0]["female_officer_ct"])
        male.append(response["results"][0]["male_officer_ct"])
    except:
        print("Not found")
        pass
print("------------------------")
print("Data Retrieval Complete")
print("------------------------")

Beginning Data Retrieval
------------------------
Processing record 1
Processing record 2
Processing record 3
Processing record 4
Processing record 5
Processing record 6
Processing record 7
Processing record 8
Processing record 9
------------------------
Data Retrieval Complete
------------------------


In [110]:
male

[106, 1839, 149, 540, None, 82, 45, 288, 90]

In [114]:
officer_df = pd.DataFrame({"ORI Code": ori_list, "Male Officers": male, "Female Officers" : female})
officer_df["Total Number"]=officer_df["Male Officers"]+officer_df["Female Officers"]
officer_df.head()

,ORI Code,Male Officers,Female Officers,Total Number
0,MD0020100,106.0,13.0,119.0
1,MA0130100,1839.0,283.0,2122.0
2,MI3351900,149.0,39.0,188.0
3,MN0620900,540.0,91.0,631.0
4,MS0250100,NaN,NaN,NaN


In [116]:
officer_merged_df = pd.merge(cases_merged_df, officer_df, on= "ORI Code")
officer_merged_df

,State,State Ab.,State Capital,Population,ORI Code,No. of Officers,No. of (Offenses,Simple Assault Cases,Stolen Property Cases,Rape Cases,Male Officers,Female Officers,Total Number
0,MARYLAND,MD,Annapolis,"39,174",MD0020100,NaN,NaN,162,3,5,106.0,13.0,119.0
1,MASSACHUSETTS,MA,Boston,"692,600",MA0130100,NaN,NaN,1612,169,43,1839.0,283.0,2122.0
2,MICHIGAN,MI,Lansing,"118,210",MI3351900,NaN,NaN,791,22,17,149.0,39.0,188.0
3,MINNESOTA,MN,Saint Paul,"308,096",MN0620900,NaN,NaN,984,43,16,540.0,91.0,631.0
4,MISSISSIPPI,MS,Jackson,"160,628",MS0250100,NaN,NaN,445,106,8,NaN,NaN,NaN
5,MISSOURI,MO,Jefferson City,"42,838",MO0260100,NaN,NaN,260,0,3,82.0,6.0,88.0
6,MONTANA,MT,Helena,"32,315",MT0250100,NaN,NaN,239,2,5,45.0,7.0,52.0
7,NEBRASKA,NE,Lincoln,"289,102",NB0550100,NaN,NaN,1301,55,19,288.0,56.0,344.0
8,NEVADA,NV,Carson City,"55,916",NV0130000,NaN,NaN,46,17,6,90.0,5.0,95.0


In [117]:
officer_merged_df.to_csv("merged_dfs.csv",
                  encoding="utf-8", index=False, header=True)